<a href="https://colab.research.google.com/github/Kartik77/CIFAR-10/blob/master/Inception.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from torchvision import models

In [12]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)


cuda:0


In [0]:
tranform_train = transforms.Compose([
    transforms.RandomResizedCrop(299),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5)),
])



In [25]:
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=tranform_train)
#testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=tranform_test)

Files already downloaded and verified


In [0]:
num_classes = 10

In [0]:
batch_size = 16

In [0]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)

#Model

In [18]:
inception = models.inception_v3(pretrained= True)

Downloading: "https://download.pytorch.org/models/inception_v3_google-1a9a5a14.pth" to /root/.cache/torch/checkpoints/inception_v3_google-1a9a5a14.pth


In [19]:
print(inception)

Inception3(
  (Conv2d_1a_3x3): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2a_3x3): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_2b_3x3): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_3b_1x1): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (Conv2d_4a_3x3): BasicConv2d(
    (conv): Conv2d(80, 192, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, t

In [0]:
for param in inception.parameters():
  param.requires_grad = False

In [0]:
aux_in_features = inception.AuxLogits.fc.in_features
inception.AuxLogits.fc = nn.Linear(aux_in_features, num_classes)

In [29]:
for param in inception.parameters():
  if param.requires_grad:
    print(param.shape)

torch.Size([10, 768])
torch.Size([10])


In [0]:
in_features = inception.fc.in_features
inception.fc = nn.Linear(in_features, num_classes)

In [31]:
for param in inception.parameters():
  if param.requires_grad:
    print(param.shape)

torch.Size([10, 768])
torch.Size([10])
torch.Size([10, 2048])
torch.Size([10])


In [0]:
inception = inception.to(device)
loss_fn = nn.CrossEntropyLoss()
opt = optim.SGD(inception.parameters(), lr=0.01)

In [0]:
def evaluation(dataloader, model):
  total, correct = 0,0
  for data in dataloader:
    inputs,labels = data
    inputs, labels = inputs.to(device), labels.to(device)
    outputs, aux_outputs = model(inputs)
    _, pred= torch.max(outputs.data, 1)
    total+= labels.size(0)
    correct += (pred==labels).sum().item()

  return 100* correct/ total

In [0]:
import copy

In [41]:
loss_epoch_arr = []
max_epochs = 1
min_loss =1000

n_iters = np.ceil(50000/batch_size)

for epoch in range(max_epochs):
  for i, data in enumerate(trainloader, 0):

    inputs, labels = data
    inputs,labels = inputs.to(device), labels.to(device)

    opt.zero_grad()

    outputs, aux_outputs = inception(inputs)
    loss = loss_fn(outputs, labels) + 0.3* loss_fn(aux_outputs, labels)
    loss.backward()
    opt.step()
     
    if min_loss> loss.item():
      min_loss = loss.item()
      best_model = copy.deepcopy(inception.state_dict())
      print('Min Loss %0.2f'% min_loss)
    del inputs, labels, outputs
    torch.cuda.empty_cache()

    if i%100 == 0:
      print('Iteration: %d/%d, Loss: %0.2f' % (i, n_iters,loss.item()))

  loss_epoch_arr.append(loss.item())
  print('Epoch : %d/%d, Train acc: %0.2f'% (epoch, max_epochs, evaluation(trainloader,inception)))

plt.plot(loss_epoch_arr)
plt.show()

Min Loss 3.17
Iteration: 0/3125, Loss: 3.17
Min Loss 2.95
Min Loss 2.94
Min Loss 2.84
Min Loss 2.82
Min Loss 2.76
Min Loss 2.74
Min Loss 2.68
Min Loss 2.61
Min Loss 2.60
Min Loss 2.58
Min Loss 2.57
Iteration: 100/3125, Loss: 2.78
Min Loss 2.48
Min Loss 2.42
Min Loss 2.30
Min Loss 2.26
Iteration: 200/3125, Loss: 2.39
Min Loss 2.18
Min Loss 2.13
Min Loss 2.13
Min Loss 2.12
Min Loss 1.92
Iteration: 300/3125, Loss: 2.17
Min Loss 1.88
Min Loss 1.85
Iteration: 400/3125, Loss: 2.60
Min Loss 1.82
Iteration: 500/3125, Loss: 2.44
Min Loss 1.78
Min Loss 1.67
Min Loss 1.60
Min Loss 1.52
Iteration: 600/3125, Loss: 2.43
Iteration: 700/3125, Loss: 2.34
Iteration: 800/3125, Loss: 2.11
Min Loss 1.50
Iteration: 900/3125, Loss: 1.95
Min Loss 1.39
Iteration: 1000/3125, Loss: 2.06
Iteration: 1100/3125, Loss: 2.00
Iteration: 1200/3125, Loss: 1.73
Min Loss 1.35
Iteration: 1300/3125, Loss: 1.98
Min Loss 1.23
Iteration: 1400/3125, Loss: 1.84
Iteration: 1500/3125, Loss: 2.36
Iteration: 1600/3125, Loss: 2.48
Ite

AttributeError: ignored

In [44]:
inception.load_state_dict(best_model)
print(evaluation(trainloader,inception))

47.664
